Imports

In [1]:
import h5py
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

Chargement du dataset

In [ ]:
dataset_simple_path = 'D:/Data/data/dog_dataset_no_aug.h5'

Classe pour charger le dataset no_aug

In [ ]:
class DogBreedDataset(Dataset):
    def __init__(self, dataset_simple_path, transform=None):
        self.file_path = dataset_simple_path
        self.transform = transform
        
        # Charger uniquement les dimensions
        with h5py.File(self.file_path, "r") as f:
            if "images" in f:
                self.data_len = f["images"].shape[0] # type: ignore
            else:
                raise KeyError("Dataset 'images' not found in the HDF5 file.")
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        with h5py.File(self.file_path, "r") as f:
            image = f["images"][idx] / 255.0  # type: ignore # Normalisation
            label = f["labels"][idx] # type: ignore
        
        # Appliquer une transformation éventuelle
        if self.transform:
            image = self.transform(image)

        # Format PyTorch
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

# Initialisation des datasets
dataset = DogBreedDataset(dataset_simple_path)

# Division en ensembles
from sklearn.model_selection import train_test_split

indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
from torch.utils.data import Subset

train_loader = DataLoader(Subset(dataset, train_indices), batch_size=32, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=32, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=32, shuffle=False)


Creation du modele

In [30]:
import torch.nn as nn
import torch.nn.functional as F

class DogBreedCNN(nn.Module):
    def __init__(self, num_classes=120):
        super(DogBreedCNN, self).__init__()

        # Couches de convolution
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)

        # Pooling et dropout
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        #print(f"Shape after conv1: {x.shape}")
        x = self.pool(F.relu(self.conv2(x)))
        #print(f"Shape after conv2: {x.shape}")
        x = self.pool(F.relu(self.conv3(x)))
        #print(f"Shape after conv3: {x.shape}")
        x = self.pool(F.relu(self.conv4(x)))
        #print(f"Shape after conv4: {x.shape}")


        # Flatten the output
        x = x.view(x.size(0), -1)
        print(f"Shape after flattening: {x.shape}")

        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

model = DogBreedCNN(num_classes=120)

In [32]:
images, labels = next(iter(train_loader))
outputs = model(images)
print(f"Model outputs shape: {outputs.shape}")

Shape after flattening: torch.Size([32, 50176])
Model outputs shape: torch.Size([32, 120])


Entrainement no aug

In [ ]:
for images, labels in train_loader:
    print(f"Images shape: {images.shape}")
    print(f"Sample images shape: {images.shape}")
    print(f"Sample labels: {labels}")
    break

Images shape: torch.Size([32, 3, 224, 224])


In [26]:
import torch.optim as optim

# Configurations
learning_rate = 0.0001
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: t

Evaluation du modele

In [29]:
from sklearn.metrics import classification_report

# Evaluation
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Rapport de classification
print(classification_report(y_true, y_pred, zero_division=1))

Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: torch.Size([32, 32, 112, 112])
Shape after conv2: torch.Size([32, 64, 56, 56])
Shape after conv3: torch.Size([32, 128, 28, 28])
Shape after conv4: torch.Size([32, 256, 14, 14])
Shape after conv1: t